In [1]:
import os
import glob
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from pyspark.sql.functions import col, month, year

import model_inference


In [2]:
# Build a .py script that takes a snapshot date, loads a model artefact and make an inference and save to datamart

## set up pyspark session

In [3]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/01 05:45:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/01 05:45:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/01 05:45:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## set up config

In [4]:
snapshot_date_str = "2008-01-01"
model_name = "diabetes_LRmodel_2009_01_01.pkl"


In [5]:
config = {}
config["snapshot_date_str"] = snapshot_date_str
config["snapshot_date"] = datetime.strptime(config["snapshot_date_str"], "%Y-%m-%d")
config["model_name"] = model_name
config["model_bank_directory"] = "model_bank/"
config["model_artefact_filepath"] = config["model_bank_directory"] + config["model_name"]

pprint.pprint(config)

{'model_artefact_filepath': 'model_bank/diabetes_LRmodel_2009_01_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'diabetes_LRmodel_2009_01_01.pkl',
 'snapshot_date': datetime.datetime(2008, 1, 1, 0, 0),
 'snapshot_date_str': '2008-01-01'}


## load model artefact from model bank

In [6]:
# Load the model from the pickle file
with open(config["model_artefact_filepath"], 'rb') as file:
    model_artefact = pickle.load(file)

print("Model loaded successfully! " + config["model_artefact_filepath"])

Model loaded successfully! model_bank/diabetes_LRmodel_2009_01_01.pkl


## load feature store

In [7]:
feature_location = "datamart/gold/feature_store/"

files_list = [feature_location + os.path.basename(f) for f in glob.glob(os.path.join(feature_location, '*'))]
feature_df = spark.read.option("header", "true").parquet(*files_list)
print("Feature DF row_count:", feature_df.count())

snapshot_date = datetime.strptime(config["snapshot_date_str"], "%Y-%m-%d")
target_year = snapshot_date.year
target_month = snapshot_date.month

# extract feature store
features_sdf = feature_df.filter((year(col('snapshot_date')) == target_year) & (month(col('snapshot_date')) == target_month))
print("extracted features_sdf", features_sdf.count(), config["snapshot_date"])

features_pdf = features_sdf.toPandas()
features_pdf

Feature DF row_count: 101766


extracted features_sdf 837 2008-01-01 00:00:00


,encounter_id,race_AfricanAmerican,race_Caucasian,race_Asian,race_Hispanic,is_female,age_midpoint,admission_severity_score,admission_source_risk_score,poor_glucose_control,metformin_ord,insulin_ord,diabetesMed,severity_x_visits,medication_density,diag_1_category_ohe,diag_2_category_ohe,diag_3_category_ohe,snapshot_date
0,20517684,0,1,0,0,1,75,0,1,0,0,0,1,0,0.016949,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2008-01-01
1,20522484,0,1,0,0,1,45,1,1,1,1,1,1,0,0.022173,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2008-01-02
2,20531412,0,1,0,0,0,45,3,2,0,0,1,1,0,0.010695,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2008-01-03
3,20539254,0,1,0,0,0,65,2,1,1,0,1,1,0,0.019231,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",2008-01-04
4,20562732,0,1,0,0,1,75,2,1,0,0,0,0,0,0.047720,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2008-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,399019610,0,1,0,0,1,85,2,2,0,0,2,1,4,0.045363,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2008-01-27
833,399027464,0,1,0,0,0,75,3,2,0,0,1,1,0,0.032258,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2008-01-28
834,399030320,0,1,0,0,1,65,1,1,0,1,0,1,4,0.093750,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2008-01-29
835,399033230,0,1,0,0,0,85,2,3,0,0,0,0,1,0.051429,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2008-01-30


## preprocess data for modeling

In [8]:
# prepare X_inference
feature_cols = [c for c in features_pdf.columns if c not in ["encounter_id", "snapshot_date", "label", "medical_specialty", 'diag_1', 'diag_2', 'diag_3']]

X_inference = features_pdf[feature_cols]

# apply transformer - standard scaler
transformer_stdscaler = model_artefact["preprocessing_transformers"]["stdscaler"]
X_inference = transformer_stdscaler.transform(X_inference)

print('X_inference', X_inference.shape[0])
X_inference

X_inference 837


array([[ 0.56493366, -2.03558834, -0.88298809, ...,  1.        ,
         0.        ,  1.        ],
       [-1.32058117, -1.08489871, -0.88298809, ...,  1.        ,
         1.        ,  1.        ],
       [-1.32058117,  0.81648056,  0.53543901, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.06357128, -1.08489871, -0.88298809, ...,  1.        ,
         0.        ,  1.        ],
       [ 1.1934386 , -0.13420907,  1.9538661 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.06357128, -1.08489871, -0.88298809, ...,  0.        ,
         0.        ,  1.        ]], shape=(837, 14))

## model prediction inference

In [10]:
# load model
model = model_artefact["model"]

# predict model
y_inference = model.predict_proba(X_inference)[:, 1]

# prepare output
y_inference_pdf = features_pdf[["encounter_id", "snapshot_date"]].copy()
y_inference_pdf["model_name"] = config["model_name"]
y_inference_pdf["model_predictions"] = y_inference
y_inference_pdf

,encounter_id,snapshot_date,model_name,model_predictions
0,20517684,2008-01-01,diabetes_LRmodel_2009_01_01.pkl,0.472177
1,20522484,2008-01-02,diabetes_LRmodel_2009_01_01.pkl,0.398734
2,20531412,2008-01-03,diabetes_LRmodel_2009_01_01.pkl,0.460277
3,20539254,2008-01-04,diabetes_LRmodel_2009_01_01.pkl,0.461612
4,20562732,2008-01-05,diabetes_LRmodel_2009_01_01.pkl,0.426232
...,...,...,...,...
832,399019610,2008-01-27,diabetes_LRmodel_2009_01_01.pkl,0.577573
833,399027464,2008-01-28,diabetes_LRmodel_2009_01_01.pkl,0.491903
834,399030320,2008-01-29,diabetes_LRmodel_2009_01_01.pkl,0.526955
835,399033230,2008-01-30,diabetes_LRmodel_2009_01_01.pkl,0.454797


## save model inference to datamart gold table

In [11]:
# create bronze datalake
gold_directory = f"datamart/gold/model_predictions/{config["model_name"][:-4]}/"
print(gold_directory)

if not os.path.exists(gold_directory):
    os.makedirs(gold_directory)

# save gold table - IRL connect to database to write
partition_name = config["model_name"][:-4] + "_predictions_" + snapshot_date_str.replace('-','_') + '.parquet'
filepath = gold_directory + partition_name
spark.createDataFrame(y_inference_pdf).write.mode("overwrite").parquet(filepath)
# df.toPandas().to_parquet(filepath,
#           compression='gzip')
print('saved to:', filepath)

datamart/gold/model_predictions/diabetes_LRmodel_2009_01_01/


saved to: datamart/gold/model_predictions/diabetes_LRmodel_2009_01_01/diabetes_LRmodel_2009_01_01_predictions_2008_01_01.parquet


In [12]:
folder_path = "datamart/gold/model_predictions/diabetes_LRmodel_2009_01_01/"
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
df = spark.read.option("header", "true").parquet(*files_list)
print("row_count:",df.count())

df.show()

row_count: 837
+------------+-------------+--------------------+-------------------+
|encounter_id|snapshot_date|          model_name|  model_predictions|
+------------+-------------+--------------------+-------------------+
|   303134546|   2008-01-16|diabetes_LRmodel_...|0.48436837244886916|
|   303137078|   2008-01-17|diabetes_LRmodel_...| 0.4596163414700474|
|   303146720|   2008-01-18|diabetes_LRmodel_...| 0.4872451202800005|
|   303152660|   2008-01-19|diabetes_LRmodel_...| 0.5250578781376238|
|   303152948|   2008-01-20|diabetes_LRmodel_...| 0.4765028298600108|
|   303156032|   2008-01-21|diabetes_LRmodel_...| 0.4894028510133471|
|   303156074|   2008-01-22|diabetes_LRmodel_...| 0.5147290389645935|
|   303166952|   2008-01-23|diabetes_LRmodel_...| 0.4364486670571307|
|   303169964|   2008-01-24|diabetes_LRmodel_...|0.36094140599766933|
|   303170570|   2008-01-25|diabetes_LRmodel_...|0.48704385906734343|
|   303174818|   2008-01-26|diabetes_LRmodel_...|0.45287369281547657|
|   3